In [20]:
import pandas as pd
import featuretools as ft # type: ignore

In [21]:
# load data from ecommerce.zip
customers = pd.read_csv('Ecommerce/Customers.csv')
products = pd.read_csv('Ecommerce/Products.csv')
sellers = pd.read_csv('Ecommerce/Seller.csv')
orders = pd.read_csv('Ecommerce/Orders.csv')

In [22]:
customers.head()

,CustomerID,CustomerName,CustomerEmail,CustomerSignupDate,CustomerCity,CustomerState
0,1001,Nollie Stone,nollie.stone@alice.it,15/4/2024 10:41,Baltimore,Maryland
1,1002,Ignacius Mainwaring,ignacius.mainwaring@aol.com,3/6/2023 4:55,Pittsburgh,Pennsylvania
2,1003,Fred Hobbert,fred.hobbert@hotmail.co.uk,5/11/2021 3:49,Houston,Texas
3,1004,Pavlov Demcik,pavlov.demcik@gmail.com,17/2/2021 22:23,Norfolk,Virginia
4,1005,Roslyn Asch,roslyn.asch@rediffmail.com,25/4/2022 18:34,El Paso,Texas


In [23]:
sellers.head()

,SellerID,SellerName,SellerEmail,SellerSignupDate,SellerCity,SellerState
0,2000,Normie Vaggs,normie.vaggs@msn.com,1/12/2018 10:48,Columbus,Ohio
1,2001,Marielle Linskill,marielle.linskill@yahoo.com,7/10/2017 6:47,Cleveland,Ohio
2,2002,Kalinda Takos,kalinda.takos@gmail.com,23/11/2020 1:04,Newport News,Virginia
3,2003,Lind Maron,lind.maron@yahoo.fr,27/8/2019 8:37,Philadelphia,Pennsylvania
4,2004,Riley Lindwall,riley.lindwall@aol.com,4/8/2021 10:08,Lawrenceville,Georgia


In [24]:
# select products wehrer product_id is in orders
products = products[products['ProductID'].isin(orders['ProductID'])]
products.head()

,ProductID,ProductName,ProductCategory,ProductPrice,SellerID
0,1,Wireless Mouse,Electronics,25.99,2000
1,2,Bluetooth Headphones,Audio,79.99,2004
4,5,LED Desk Lamp,Home,35.00,2009
5,6,Mechanical Keyboard,Computers,99.99,2005
6,7,Portable SSD,Storage,129.99,2000


In [25]:
orders.head()

,OrderID,OrderDate,CustomerID,OrderQuantity,ProductID,ShipDate
0,de7ca4ca-30e9-8751-91cb-1821bcd8f394,24/2/2024 21:34,1007,3,59,24/2/2024 21:34
1,44699261-d1c9-8bbb-646e-bf707f6e8426,31/5/2024 6:58,1004,6,74,31/5/2024 6:58
2,6a526fbc-c38b-0e0c-57ae-27391d520865,25/7/2024 11:12,1009,7,16,25/7/2024 11:12
3,fca1e781-112f-0f69-a041-c239c37e5a79,25/12/2023 9:39,1010,2,5,25/12/2023 9:39
4,7ec5dcfc-b781-2de7-3a73-1e89ffc5c67b,17/11/2023 13:44,1019,1,40,17/11/2023 13:44


In [26]:
orders = orders.merge(products[['ProductID', 'SellerID','ProductPrice']], on='ProductID', how='left')
orders['OrderTotal'] = orders['ProductPrice'] * orders['OrderQuantity']
orders.head()


,OrderID,OrderDate,CustomerID,OrderQuantity,ProductID,ShipDate,SellerID,ProductPrice,OrderTotal
0,de7ca4ca-30e9-8751-91cb-1821bcd8f394,24/2/2024 21:34,1007,3,59,24/2/2024 21:34,2001,39.99,119.97
1,44699261-d1c9-8bbb-646e-bf707f6e8426,31/5/2024 6:58,1004,6,74,31/5/2024 6:58,2003,89.99,539.94
2,6a526fbc-c38b-0e0c-57ae-27391d520865,25/7/2024 11:12,1009,7,16,25/7/2024 11:12,2003,39.99,279.93
3,fca1e781-112f-0f69-a041-c239c37e5a79,25/12/2023 9:39,1010,2,5,25/12/2023 9:39,2009,35.00,70.00
4,7ec5dcfc-b781-2de7-3a73-1e89ffc5c67b,17/11/2023 13:44,1019,1,40,17/11/2023 13:44,2006,27.99,27.99


In [27]:
print(orders.info())
print(customers.info())
print(products.info())
print(sellers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OrderID        100 non-null    object 
 1   OrderDate      100 non-null    object 
 2   CustomerID     100 non-null    int64  
 3   OrderQuantity  100 non-null    int64  
 4   ProductID      100 non-null    int64  
 5   ShipDate       100 non-null    object 
 6   SellerID       100 non-null    int64  
 7   ProductPrice   100 non-null    float64
 8   OrderTotal     100 non-null    float64
dtypes: float64(2), int64(4), object(3)
memory usage: 7.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CustomerID          20 non-null     int64 
 1   CustomerName        20 non-null     object
 2   CustomerEmail       20 non-null     object
 3  

In [28]:
print('orders:', orders.duplicated().sum())
print('products:', products.duplicated().sum())
print('customers:', customers.duplicated().sum())
print('payments:', sellers.duplicated().sum())

orders: 0
products: 0
customers: 0
payments: 0


In [29]:

# create an entity set
es = ft.EntitySet('orders')

es.add_dataframe(dataframe_name="orders",
                 dataframe=orders,
                 index="OrderID",
                 time_index="OrderDate")

es.add_dataframe(dataframe_name="customer",
                 dataframe=customers,
                 index="CustomerID",
                 time_index="CustomerSignupDate")

es.add_dataframe(dataframe_name= 'products',
                 dataframe= products,
                 index= 'ProductID')

es.add_dataframe(dataframe_name= 'sellers',
                 dataframe= sellers,
                 index= 'SellerID')   

es.add_relationship("products", "ProductID", "orders", "ProductID")
es.add_relationship("customer", "CustomerID", "orders", "CustomerID")
es.add_relationship("sellers", "SellerID", "orders", "SellerID")

c:\Users\Aqilah\miniconda3\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pd.to_datetime(
c:\Users\Aqilah\miniconda3\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\Aqilah\miniconda3\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\Aqilah\miniconda3\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `

Entityset: orders
  DataFrames:
    orders [Rows: 100, Columns: 9]
    customer [Rows: 20, Columns: 6]
    products [Rows: 67, Columns: 5]
    sellers [Rows: 10, Columns: 6]
  Relationships:
    orders.ProductID -> products.ProductID
    orders.CustomerID -> customer.CustomerID
    orders.SellerID -> sellers.SellerID

In [30]:
# deep feature synthesis
features, feature_defs = ft.dfs(entityset=es,
                                target_dataframe_name="orders",
                                agg_primitives=["sum", "mean", "mode",'count'])

c:\Users\Aqilah\miniconda3\Lib\site-packages\featuretools\synthesis\dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['mode']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)
c:\Users\Aqilah\miniconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x0000018172621080> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
c:\Users\Aqilah\miniconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculat

In [31]:
feature_defs

[<Feature: CustomerID>,
 <Feature: OrderQuantity>,
 <Feature: ProductID>,
 <Feature: SellerID>,
 <Feature: ProductPrice>,
 <Feature: OrderTotal>,
 <Feature: DAY(OrderDate)>,
 <Feature: DAY(ShipDate)>,
 <Feature: MONTH(OrderDate)>,
 <Feature: MONTH(ShipDate)>,
 <Feature: WEEKDAY(OrderDate)>,
 <Feature: WEEKDAY(ShipDate)>,
 <Feature: YEAR(OrderDate)>,
 <Feature: YEAR(ShipDate)>,
 <Feature: products.ProductPrice>,
 <Feature: products.SellerID>,
 <Feature: products.COUNT(orders)>,
 <Feature: products.MEAN(orders.OrderQuantity)>,
 <Feature: products.MEAN(orders.OrderTotal)>,
 <Feature: products.MEAN(orders.ProductPrice)>,
 <Feature: products.SUM(orders.OrderQuantity)>,
 <Feature: products.SUM(orders.OrderTotal)>,
 <Feature: products.SUM(orders.ProductPrice)>,
 <Feature: customer.COUNT(orders)>,
 <Feature: customer.MEAN(orders.OrderQuantity)>,
 <Feature: customer.MEAN(orders.OrderTotal)>,
 <Feature: customer.MEAN(orders.ProductPrice)>,
 <Feature: customer.SUM(orders.OrderQuantity)>,
 <Featur

In [32]:
features.head()

,CustomerID,OrderQuantity,ProductID,SellerID,ProductPrice,OrderTotal,DAY(OrderDate),DAY(ShipDate),MONTH(OrderDate),MONTH(ShipDate),...,customer.MONTH(CustomerSignupDate),customer.WEEKDAY(CustomerSignupDate),customer.YEAR(CustomerSignupDate),sellers.COUNT(orders),sellers.MEAN(orders.OrderQuantity),sellers.MEAN(orders.OrderTotal),sellers.MEAN(orders.ProductPrice),sellers.SUM(orders.OrderQuantity),sellers.SUM(orders.OrderTotal),sellers.SUM(orders.ProductPrice)
OrderID,,,,,,,,,,,,,,,,,,,,,
7cbbf5fe-817c-4afb-2680-2dd6f6f809af,1017,1,79,2009,59.99,59.99,2,2,11,11,...,4,1,2022,14,4.285714,260.244286,62.347857,60.0,3643.42,872.87
3d3612ab-2208-0e08-9b47-730d63e6b837,1013,7,96,2002,34.99,244.93,4,4,11,11,...,2,5,2024,11,4.818182,132.213636,30.259091,53.0,1454.35,332.85
4a3134c7-3fcd-8f7d-217f-751c2b4beab9,1001,1,87,2001,12.99,12.99,6,6,11,11,...,4,0,2024,7,4.142857,271.101429,55.418571,29.0,1897.71,387.93
395fb393-cd65-1f5a-81a6-1b1d68480133,1013,8,46,2005,14.99,119.92,6,6,11,11,...,2,5,2024,11,4.727273,254.966364,61.264545,52.0,2804.63,673.91
55b38080-73b4-2fa3-f68b-c8637b3e04cd,1010,7,38,2007,29.99,209.93,7,7,11,11,...,7,0,2019,9,7.444444,413.390000,53.214444,67.0,3720.51,478.93


In [ ]:
features.to_csv('features.csv', index=False)

In [34]:
# sected features
selected_features = ["WEEKDAY(OrderDate)", "YEAR(OrderDate)", "MONTH(OrderDate)", "MONTH(ShipDate)", "WEEKDAY(ShipDate)", "YEAR(ShipDate)"]

# Filter feature matrix for selected features
merge_features = features[selected_features]
orders = orders.merge(merge_features, left_on="OrderID", right_index=True)

orders.head()

,OrderID,OrderDate,CustomerID,OrderQuantity,ProductID,ShipDate,SellerID,ProductPrice,OrderTotal,WEEKDAY(OrderDate),YEAR(OrderDate),MONTH(OrderDate),MONTH(ShipDate),WEEKDAY(ShipDate),YEAR(ShipDate)
7cbbf5fe-817c-4afb-2680-2dd6f6f809af,7cbbf5fe-817c-4afb-2680-2dd6f6f809af,2023-11-02 01:28:00,1017,1,79,2023-11-02 01:28:00,2009,59.99,59.99,3,2023,11,11,3,2023
3d3612ab-2208-0e08-9b47-730d63e6b837,3d3612ab-2208-0e08-9b47-730d63e6b837,2023-11-04 08:53:00,1013,7,96,2023-11-04 08:53:00,2002,34.99,244.93,5,2023,11,11,5,2023
4a3134c7-3fcd-8f7d-217f-751c2b4beab9,4a3134c7-3fcd-8f7d-217f-751c2b4beab9,2023-11-06 02:47:00,1001,1,87,2023-11-06 02:47:00,2001,12.99,12.99,0,2023,11,11,0,2023
395fb393-cd65-1f5a-81a6-1b1d68480133,395fb393-cd65-1f5a-81a6-1b1d68480133,2023-11-06 05:50:00,1013,8,46,2023-11-06 05:50:00,2005,14.99,119.92,0,2023,11,11,0,2023
55b38080-73b4-2fa3-f68b-c8637b3e04cd,55b38080-73b4-2fa3-f68b-c8637b3e04cd,2023-11-07 09:06:00,1010,7,38,2023-11-07 09:06:00,2007,29.99,209.93,1,2023,11,11,1,2023


In [35]:
# sected features
selected_features = ["CustomerID","customer.COUNT(orders)","customer.SUM(orders.OrderQuantity)", "customer.MEAN(orders.OrderTotal)"]

# Filter feature matrix for selected features
merge_features = features[selected_features]
merge_features = merge_features.groupby("CustomerID").mean()
customers = customers.merge(merge_features, left_on="CustomerID", right_index=True)
customers.sort_values("CustomerID")
customers.head()

,CustomerID,CustomerName,CustomerEmail,CustomerSignupDate,CustomerCity,CustomerState,customer.COUNT(orders),customer.SUM(orders.OrderQuantity),customer.MEAN(orders.OrderTotal)
1011,1011,Julissa Mapowder,julissa.mapowder@orange.fr,2017-05-27 22:23:00,Des Moines,Iowa,7.0,38.0,527.244286
1008,1008,Abagael Rosevear,abagael.rosevear@gmail.com,2019-05-23 10:13:00,Omaha,Nebraska,5.0,30.0,339.558000
1010,1010,Emera Whelband,emera.whelband@msn.com,2019-07-08 00:58:00,Laredo,Texas,7.0,39.0,316.375714
1009,1009,Marten Kornilyev,marten.kornilyev@yahoo.com,2019-07-11 19:19:00,White Plains,New York,4.0,22.0,379.945000
1014,1014,Crysta Howson,crysta.howson@hotmail.fr,2020-08-09 16:32:00,Baton Rouge,Louisiana,3.0,12.0,185.960000


In [36]:
# sected features
selected_features = ["SellerID", "sellers.COUNT(orders)", "sellers.SUM(orders.OrderQuantity)", "sellers.SUM(orders.OrderTotal)"]

# Filter feature matrix for selected features
merge_features = features[selected_features]
merge_features = merge_features.groupby("SellerID").mean()
sellers = sellers.merge(merge_features, left_on="SellerID", right_index=True)
sellers.sort_values("SellerID")
sellers.head()

,SellerID,SellerName,SellerEmail,SellerSignupDate,SellerCity,SellerState,sellers.COUNT(orders),sellers.SUM(orders.OrderQuantity),sellers.SUM(orders.OrderTotal)
2000,2000,Normie Vaggs,normie.vaggs@msn.com,1/12/2018 10:48,Columbus,Ohio,3.0,15.0,1145.85
2001,2001,Marielle Linskill,marielle.linskill@yahoo.com,7/10/2017 6:47,Cleveland,Ohio,7.0,29.0,1897.71
2002,2002,Kalinda Takos,kalinda.takos@gmail.com,23/11/2020 1:04,Newport News,Virginia,11.0,53.0,1454.35
2003,2003,Lind Maron,lind.maron@yahoo.fr,27/8/2019 8:37,Philadelphia,Pennsylvania,9.0,58.0,6123.42
2004,2004,Riley Lindwall,riley.lindwall@aol.com,4/8/2021 10:08,Lawrenceville,Georgia,21.0,101.0,6328.99


In [37]:
# sected features
selected_features = ["ProductID","products.SUM(orders.OrderQuantity)","products.SUM(orders.OrderTotal)"]

# Filter feature matrix for selected features
merge_features = features[selected_features]
merge_features = merge_features.groupby("ProductID").mean()
products = products.merge(merge_features, left_on="ProductID", right_index=True)
products.sort_values("ProductName")
products.head()

,ProductID,ProductName,ProductCategory,ProductPrice,SellerID,products.SUM(orders.OrderQuantity),products.SUM(orders.OrderTotal)
1,1,Wireless Mouse,Electronics,25.99,2000,6.0,155.94
2,2,Bluetooth Headphones,Audio,79.99,2004,12.0,959.88
5,5,LED Desk Lamp,Home,35.00,2009,2.0,70.00
6,6,Mechanical Keyboard,Computers,99.99,2005,3.0,299.97
7,7,Portable SSD,Storage,129.99,2000,7.0,909.93


In [38]:
# orders.to_csv('Data/orders_transformed.csv', index=False)
# customers.to_csv('Data/customers_transformed.csv', index=False)
# sellers.to_csv('Data/sellers_transformed.csv', index=False)
# products.to_csv('Data/products_transformed.csv', index=False)